In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [ ]:
class CompoundScaling:

In [ ]:
unet

In [ ]:
resnet.encoder

In [6]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UNet


In [7]:
from glasses.nn.models.classification.efficientnet import EfficientNet, MemoryEfficientSwish
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_dst  = EfficientNet(activation=MemoryEfficientSwish)

summary(model_dst.to(device), (3,224,224))

model_dst

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
MemoryEfficientSwish-3         [-1, 32, 112, 112]               0
   DepthWiseConv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
MemoryEfficientSwish-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
MemoryEfficientSwish-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
        ChannelSE-12         [-1, 32, 112, 112]               0
        Conv2dPad-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1,

EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): ConvBnAct(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (act): MemoryEfficientSwish()
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): EfficientNetBasicBlock(
            (block): Residual(
              (block): Sequential(
                (depth): ConvBnAct(
                  (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                  (bn): BatchNorm2d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (act): MemoryEfficientSwish()
                )
                (point): Sequential(
                  (0): ChannelSE(
                    (avg_pool): AdaptiveAvgPool2d(output_size=1)
                    (att): Sequential(
    

In [81]:
from dataclasses import dataclass
import numpy as np
import math
from glasses.nn.models.classification.efficientnet import EfficientNet, MemoryEfficientSwish
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

arch_params = {
    # arch width_multi depth_multi input_h dropout_rate
    'b0': (1.0, 1.0, 224, 0.2),
    'b1': (1.0, 1.1, 240, 0.2),
    'b2': (1.1, 1.2, 260, 0.3),
    'b3': (1.2, 1.4, 300, 0.3),
    'b4': (1.4, 1.8, 380, 0.4),
    'b5': (1.6, 2.2, 456, 0.4),
    'b6': (1.8, 2.6, 528, 0.5),
    'b7': (2.0, 3.1, 600, 0.5),
}

def make_divisible(value, divisor=8):
    new_value = max(divisor, int(value + divisor / 2) // divisor * divisor)
    if new_value < 0.9 * value:
        new_value += divisor
    return new_value

def width_scaling(width, width_multi):
    scaled = width if width == 1 else int(make_divisible(width * width_multi))
    return scaled

def depth_scaling(depth, depth_multi):
    scaled = depth if depth_multi == 1 else int(math.ceil(depth_multi * depth))
    return scaled

scaling_params = {
    # name : (alpha, beta)
    'b0': (1.0, 1.0),
    'b1': (1.1, 1.0),
    'b2': (1.2, 1.1),
    'b3': (1.4, 1.2),
    'b4': (1.8, 1.4),
    'b5': (2.2, 1.6),
    'b6': (2.6, 1.8),
    'b7': (3.1, 2.0),
    'b8': (3.6, 2.2),
    'l2': (5.3, 4.3)
}

@dataclass
class CompoundScaler:
    alpha: float = 1.2
    beta: float = 1.1
    
    def __call__(self, model, depths, widths, *args, **kwargs):
        
        depths = [depth_scaling(d, self.alpha) for d in depths]
        widths = [width_scaling(w, self.beta) for w in widths]
        
        return model(*args, depths=depths, widths=widths, **kwargs)


In [80]:
summary(EfficientNet.b0().to(device)

1.0 1.0


EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): ConvBnAct(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (act): Swish()
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): EfficientNetBasicBlock(
            (block): Residual(
              (block): Sequential(
                (depth): ConvBnAct(
                  (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                  (bn): BatchNorm2d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (act): Swish()
                )
                (point): Sequential(
                  (0): ChannelSE(
                    (avg_pool): AdaptiveAvgPool2d(output_size=1)
                    (att): Sequential(
                      (conv1): Con

In [30]:
model  = EfficientNet(depths=depths, widths=widths)

summary(model.to(device), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             Swish-3         [-1, 32, 112, 112]               0
   DepthWiseConv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             Swish-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
             Swish-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
        ChannelSE-12         [-1, 32, 112, 112]               0
        Conv2dPad-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [31]:
from efficientnet_pytorch import EfficientNet


model = EfficientNet.from_pretrained('efficientnet-b1')
summary(model.to(device), (3,224,224))

Loaded pretrained weights for efficientnet-b1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [ ]:
model_dst

In [ ]:
y_dst

tensor([[-9.3545e-01,  4.4536e-01,  4.2809e-02, -7.1374e-01,  3.6706e-01,
          2.3940e-01, -1.3259e+00, -3.2074e-01, -1.6859e+00, -7.2495e-01,
         -1.2006e+00, -5.1331e-01, -7.6875e-01, -1.4668e+00, -8.2695e-01,
         -4.0084e-02,  3.0943e-01, -7.2798e-01, -4.7880e-01, -8.8295e-01,
         -1.6215e+00,  7.9493e-01,  3.6127e-02, -5.1066e-01, -4.0636e-01,
         -8.5400e-01, -1.2117e+00, -2.1563e-01, -1.1900e+00, -5.3588e-01,
         -9.6766e-01, -8.8796e-01, -1.2135e+00, -6.0439e-01, -7.0122e-01,
         -1.4644e+00, -1.1432e+00, -9.4588e-01, -9.1610e-01, -1.1081e+00,
         -7.2389e-01, -8.2417e-01, -8.6236e-01, -1.3655e+00, -1.6005e+00,
         -1.3012e-01, -1.6388e+00, -7.9500e-01, -1.2640e+00, -1.1749e+00,
         -1.2268e+00, -4.6094e-01, -7.8901e-01, -1.4785e+00, -2.4503e+00,
         -5.8356e-01, -1.4344e+00, -6.2406e-01, -1.2202e+00, -3.8103e-01,
         -1.2667e+00, -7.2553e-01, -1.5403e+00, -2.9658e-01, -7.7501e-01,
         -4.5203e-01, -1.2975e+00, -1.

In [ ]:
model_src